Aplicação de técnicas do Método Proposto
---
## Extração de Características

In [1]:
# Bibliotecas Utilizadas

import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
from sklearn.cluster import KMeans 
import skimage.color
import cv2
from scipy import misc
from skimage.color import rgb2gray
from skimage import color
import skimage.metrics
from sklearn.metrics import mean_squared_error
from PIL import Image
from scipy import ndimage
import functions as fun
import time
from skimage.feature import hog
from skimage import exposure
import pandas as pd
import seaborn as sns
import csv
from math import copysign, log10
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.impute import KNNImputer
from os import chdir, getcwd, listdir
from os.path import isfile
import os
import oci
import boto3

#### Configurações e funcionalidades dos Buckets: Oracle Cloud

In [ ]:
# Função para configurações da Cloud
def config_cloud(flag):

    # Definição de variável URL de acesso ao Endpoint Oracle Cloud
    if flag == 'p':
        # Ambiente de Produção - Config Cloud
        url = "https://grinqewrovfi.compat.objectstorage.sa-saopaulo-1.oraclecloud.com" 
    
    elif flag =='t':
        
        # Ambiente de Testes - Config Cloud
        url = "https://grwbzp0j0zza.compat.objectstorage.sa-saopaulo-1.oraclecloud.com"

    # Chamada de função para obter as configurações da Cloud
    fun.configuration_Cloud()

    # Chamada de função para obter o resource
    resource_value = fun.resource()
       
    return url, resource_value

# Chamada de função para configurar a Cloud 
url, resource_value = config_cloud('p')

#### Configurações para Processamento

In [3]:
# Armazenar todos os nomes dos arquivos de imagens do Banco de Imagens

# Definição dos "Paths"

full_path = "/home/datascience/classes-images-selected/"
path = "/home/datascience/classes-images-selected/"

# Definição do contador para automatizar
contador_imagens = 0

In [ ]:
# Função para pegar os objetos selecionados no bucket
def objects_to_bucket(flag, bucket, resource_value, url, contador_imagens):
            
    if os.path.exists(path) ==False:
        os.makedirs(path)
        
    # Imprimir os buckets encontrados
    try:
        for bucket_name in resource_value.buckets.all():
            print (bucket_name.name)
        print("\n Conexao com sucesso na cloud!!\n")
    except:
        print("Falha de conexao na cloud")
    
    files = fun.getObjectsToBucket(str(bucket), resource_value)

    # Flag 'd' - Processo de download de dados via (API) Boto3
    if flag == 'd':
        # Chamada de funções para upload de arquivo: destino 'pasta de trabalho do ambiente data science'
        fun.download_to_bucket(str(bucket), url, files, path)
        message = "Download de arquivos com sucesso!!"
    
    # Flag 's' - Processo de sincronismo de dados via (API) rclone
    elif flag == 's':
        try:
            #rclone sync source:path dest:path
            cmd = 'rclone sync ' + str(bucket) + ':' + str(bucket) + ' ' + path 
            os.system(cmd)
            message = "Arquivos sincronizados com successo!!"
        except:
            message = "Erro de sincronismo"
    
    return message, path

# Chamada de função para obter objetos selecionados do bucket
message = objects_to_bucket('s', 'BK-feature-selected', resource_value, url, contador_imagens)
print(message)

# Armazenar os arquivos de entrada para serem processados
filepaths = [f for f in os.listdir(path) if f.endswith('.jpg')]

# Imprimir o resultado dos arquivos processados
print("\n Quantidade de Arquivos Processados: " + str(len(filepaths)))

# Imprimir a posição de cada arquivo (Facilitar a localização em caso de falha de processamento)
for i in range(0,len(filepaths)):
    print("\n" + str(i) + " --> " + filepaths[i])
    

In [5]:
# Função para Seleção automática de imagens no vetor carregado
def selecao_automatica_img(filepaths, contador_imagens):
    nome_imagem = '/' + filepaths[contador_imagens]     

    # Verifica a "classe" da imagem selecionada (Classe Verde = 1 | Classe Amarela = 2 | Classe Marrom = 3)
    if 'verde' in nome_imagem:
        cor = 'verde'
        op = 1
    elif 'amarela' in nome_imagem:
        cor = 'amarela'
        op = 2
    elif 'marrom' in nome_imagem:
        cor = 'marrom'
        op= 3
        
    # Deninir o nome da pasta de acordo com a imagem 
    pasta = nome_imagem[-12::]

    path_out = 'caracteristicas_2/' + pasta + '/' + cor
    # Verifica se a pasta já foi criada. Em caso negativo - cria a pasta
    if os.path.exists(path_out) == False:
        os.makedirs(path_out) 

    # Exibição da imagem selecionada e da cor a ser processada
    print('Imagem Selecionada: ' + nome_imagem)
    print('Cor a ser processada: ' + cor)
    print('Iteração de nº: ' + str(contador_imagens))

    return op, nome_imagem, path_out

#### HOG - Histogram of Oriented Gradients

In [6]:
# Função para entrada do arquivo para análise HOG
def entrada_imagem(full_path, nome_imagem):
    image = cv2.cvtColor(cv2.imread(full_path + nome_imagem), cv2.COLOR_BGR2RGB)
    gray = color.rgb2gray(image)
    plt.figure(figsize=[15, 5])
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Imagem Original')
    plt.subplot(1, 2, 2)
    plt.imshow(gray)
    plt.title('Imagem Tons de Cinza')

    return image, gray

In [7]:
# Versão com a biblioteca "skimage.feature import hog"
# Função para Processamento HOG
def processamento_HOG(gray):
    fd, hog_image = hog(gray, orientations=8,\
                        pixels_per_cell=(8,8),\
                        cells_per_block=(8,8),\
                        visualize=True,\
                        feature_vector=True,\
                        multichannel= False,\
                        transform_sqrt=True)

    return fd, hog_image

In [8]:
# Função para plotagem da imagem HOG
def plot_imagem_HOG(image, hog_image):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 25), sharex=True, sharey=True)

    ax1.axis('off')
    ax1.imshow(image, cmap=plt.cm.gray)
    ax1.set_title('Imagem de Entrada')

    # Restruturar o histograma para melhor visualização
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 20))

    ax2.axis('off')
    ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray_r)
    ax2.set_title('Histogram of Oriented Gradients')
    plt.show()

In [9]:
# Função para armazenar o vetor de características HOG em disco
def armazenar_vetor_HOG(fd):
    text = ''
    for i in range(0,fd.size):
        if (fd[i]!=0):
            text+= '{}'.format(fd[i])
            text+= '\n'
    
    return text

In [10]:
# Função para gravar a String em arquivo
def gravar_string_arquivo(path_out,contador_imagens, url):
    with open(path_out  + '/exemploHOG' + '_' + str(contador_imagens) + '.txt', 'w') as f:
        f.write(text)
    
    # Chamada de função para updates de arquivos da 'pasta de trabalho' para o bucket selecionado       
    fun.upload_from_bucket('BK-feature-vector', url, path_out  + '/exemploHOG' + '_' + str(contador_imagens) + '.txt')

#### Exibição dos descritores HOG - Dataframe de Dados


In [11]:
# Função para carregar os dados CSV no Dataframe
def carregar_dados_dataframe(path_out, contador_imagens):
    df_hog = pd.read_csv(path_out + '/exemploHOG' + '_' + str(contador_imagens) + '.txt', sep = ',', header = None)
    # Gerar o cabeçalho para os dados CSV
    df_hog.columns =['hog']
    # Exibição do Dataframe
    df_hog

    return df_hog

#### SIFT (Scale-Invariant Feature Transform)

In [12]:
# Função para processamento do SIFT
def processar_SIFT(full_path, url, path_out, contador_imagens, nome_imagem, image):
    image = cv2.imread(full_path + nome_imagem)
    gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute (gray, None)
    image= cv2.drawKeypoints(gray,kp,image)
    image=cv2.drawKeypoints(gray,kp,image,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imwrite(path_out + '/sift_keypoints' + '_' + str(contador_imagens) + '.jpg',image)

    # Chamada de função para updates de arquivos da 'pasta de trabalho' para o bucket selecionado 
    fun.upload_from_bucket('BK-feature-vector', url, path_out + '/sift_keypoints' + '_' + str(contador_imagens) + '.jpg')

    img_sift = cv2.imread(path_out + '/sift_keypoints' + '_' + str(contador_imagens) + '.jpg')
    plt.figure(figsize=[15,15])
    plt.title('Pontos Chave - SIFT')
    plt.imshow(img_sift)

    return kp, des, sift, img_sift

In [13]:
# função para exibição dos características dados gerados SIFT
def exibir_caracateristicas_dados_SIFT():
    print(" Tamanho:" + str(des.size)+",\n Shape:" + str(des.shape) + ",\n Tipo:" + str(des.dtype))

In [14]:
# Função para armazenar o vetor de características SIFT em disco
def armazenar_vetor_SIFT():
    # Chamada da função para gravar matriz em arquivo CSV
    fun.salvarMatrizArquivoCSV(des,path_out + '/exemploVetorSIFT' + '_' + str(contador_imagens) + '.csv')
    ## Chamada de função para updates de arquivos da 'pasta de trabalho' para o bucket selecionado 
    fun.upload_from_bucket('BK-feature-vector', url, path_out + '/exemploVetorSIFT' + '_' + str(contador_imagens) + '.csv')

#### Exibição dos descritores SIFT - Dataframe de Dados


In [15]:
# Função para gerar o cabeçalho para os dados CSV
def gerar_data_header(des, path_out, contador_imagens):
    header_list = []
    lin, col = des.shape
    for i in range(0,col):
        header_list.append("SIFT"+str([i]))
        # Carrega os dados CSV no Dataframe
        df_sift = pd.read_csv(path_out + '/exemploVetorSIFT' + '_' + str(contador_imagens) + '.csv', sep = ',',header= None)
        
    # Carrega os dados de cabeçalho no Dataframe
    df_sift.columns = header_list
    #Exibição do Dataframe
    df_sift

    return df_sift 

#### Momentos Invariantes de Hu

In [16]:
# Função para processar Momentos Invariantes de Hu
def processar_momentos_HU(path_out, full_path, url, contador_imagens, nome_imagem):

    # Ler a imagem
    image = cv2.imread(full_path + nome_imagem, cv2.IMREAD_GRAYSCALE)

    # Threshold image
    _,image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)

    # Calcular os Momentos
    moments = cv2.moments(image)
    # Calcular os Momentos de Hu
    huMoments = cv2.HuMoments(moments)
    text_value=''
    
    for i in range(0,7):
        huMoments[i] = -1* copysign(1.0, huMoments[i]) * log10(abs(huMoments[i]))
        text_value += str(huMoments[i]) + '\n'
    
    # Grava a String em arquivo e Exibe os valores calculados
    with open(path_out + '/exemploMomentsHU' + '_' + str(contador_imagens) + '.txt', 'w') as f:
        f.write(text_value)
    # Chamada de função para updates de arquivos da 'pasta de trabalho' para o bucket selecionado 
    fun.upload_from_bucket('BK-feature-vector', url, path_out + '/exemploMomentsHU' + '_' + str(contador_imagens) + '.txt')

    return moments, huMoments, text_value

#### Gerar o Dataframe Momentos HU

In [17]:
# Função para tratar os dados dos Momentos HU e gerar o Dataframe
def tratar_dados_HU_dataframe(path_out, contador_imagens):
    df_hu = pd.read_csv(path_out + '/exemploMomentsHU' + '_' + str(contador_imagens) + '.txt', header= None)
    # Carrega os dados de cabeçalho no Dataframe
    df_hu.columns =['HU']
    # Tratamento dos dados
    df_hu['HU'] = df_hu['HU'].apply(lambda x: str(x).replace("[","")) # Retirada do "[" da string
    df_hu['HU'] = df_hu['HU'].apply(lambda x: str(x).replace("]","")) # Retirada do "]" da string
    # Conversão de string para Float
    df_hu['HU'] = df_hu['HU'].astype('float64')
    #Exibição dos dados Dataframe HU
    df_hu

    return df_hu

#### Concatenação dos vetores de características (HOG, SIFT e HU)

In [18]:
# Função para concatenar os dados dos dataframes de características HOG, SIFT e HU
def concatenar_dados_dataframes(df_hog, df_sift, df_hu):
    df_total = pd.concat([df_hog, df_sift, df_hu], axis=1)

    # Imprimir os dados na tela do conjunto total de dados
    df_total

    return df_total

In [19]:
# Função para gerar arquivo CSV com os dados completos
def gerar_arquivo_dados_completos(url, path_out,contador_imagens):
    df_total.to_csv(path_out + '/dados_HOG_SIFT_HU' + '_' + str(contador_imagens) + '.csv', index = None, header=True)

    # Chamada de função para updates de arquivos da 'pasta de trabalho' para o bucket selecionado 
    fun.upload_from_bucket('BK-feature-vector', url, path_out + '/dados_HOG_SIFT_HU' + '_' + str(contador_imagens) + '.csv')

#### Verificação de valores faltantes (Missing Values)

In [20]:
# Função para verificar se há colunas com valores faltantes no conjunto de dados
def verificar_valores_faltantes():
    df_total.isnull().sum()

#### Tratamento dos valores faltantes com '0'

In [21]:
# Função para preencher os dados com valores "0"
def preencher_dados_faltantes(df_total):
    df_total['HU'] = df_total['HU'].fillna(0).head(9760000) # Coluna correspondente aos Momentos de HU
    for i in range(0, 128):
        nome = "SIFT" + str([i])
        df_total[nome] = df_total[nome].fillna(0).head(9760000) # Colunas correspondentes ao SIFT
    
    return df_total

In [22]:
# Função para conferir se há valores faltantes no conjunto de dados - Pós tratamento
def conferir_valores_faltantes():
    df_total.isnull().sum()

#### Normalização dos dados completos de caracterisiticas - Preparação para entrada no PCA


In [23]:
# Função para normalização dos dados (HOG, SIFT e HU)
def normalizar_dados(df_total):
    normalized_df_total= (df_total-df_total.min())/(df_total.max()-df_total.min())
    return normalized_df_total

#### Correlação dos dados - Preparação para entrada no PCA

In [24]:
# Função para aplicar a correlação dos dados normalizados
def aplicar_correlacao(normalized_df_total):
    corr_df_total = normalized_df_total.corr()
    corr_df_total
    return corr_df_total

#### Análise dos dados correlacionados - Gráfico de Dispersão

In [25]:
# Plotagem do gráfico de dispersão dos dados correlacionados
#sns.distplot(corr_df_total)

#### Gráfico de HeatMap dos dados correlacionados

In [26]:
# Plotagem do gráfico HeatMAp
#plt.figure(figsize=[100,100])
#sns.heatmap(corr_df_total, annot= True)

#### Aplicação de PCA para redução de dimensionalidade - Vetor completo de Características (HOG, SIFT e HU)

In [27]:
# Função para aplicar PCA - Redução de Dimensionalidade
def aplicar_PCA(normalized_df_total): #corr_df_total (trocado por normalized_df_total)
    # Normaliza medidas, de forma que cada coluna de measurements_norm
    # possui média 0 e desvio padrão 1
    # measurements_norm = scale(normalized_df_total, axis=0) Retirado

    # Cria instância da classe PCA, com projeção em 2 eixos (2D), e aplica
    # o PCA nos dados
    pca_instance = PCA(n_components=10)
    pca_data = pca_instance.fit_transform(normalized_df_total)
    return pca_instance, pca_data

In [28]:
# Função para exibir os dados do Dataframe PCA
def exibir_dados_PCA(pca_data):
    df_pca = pd.DataFrame(pca_data)
    df_pca
    return df_pca

#### Gráfico de HeatMap dos dados após aplicação do PCA

In [29]:
# Plotagem do gráfico HeatMAp Após o PCA
#plt.figure(figsize=[10,25])
#sns.heatmap(df_pca, annot= True)

#### Adicionar, nos dados, uma coluna com valor correspondente à Classe de Cor Processada. Recurso para uso na etapa de Análise de Dados - Via algoritmo de Machine Learning, onde: | 1: Classe Verde | 2: Classe Amarela | 3: Classe Marrom

In [30]:
# Função para adicionar coluna no final do Dataframe com preenchimento de acordo com a Classe Processada
# 1: Classe Verde | 2: Classe Amarela | 3: Classe Marrom
def adicionar_coluna_dataframe(df_pca, op, path_out, nome_imagem, contador_imagens):
    df_pca_export = df_pca
    classe = []

    for i in range(len(df_pca_export)):
        classe.append(op)
    # 
    df_pca_export['classe'] = classe

    # Geração de arquivo CSV com os dados completos
    df_pca_export.to_csv(path_out + nome_imagem + '_' + 'dados_PCA' + '_' + str(contador_imagens) + '.csv', index = None, header=True)
    # Chamada de função para updates de arquivos da 'pasta de trabalho' para o bucket selecionado
    fun.upload_from_bucket('BK-feature-vector', url, path_out + nome_imagem + '_' + 'dados_PCA' + '_' + str(contador_imagens) + '.csv')
    df_pca_export

    return df_pca_export

In [ ]:
# Chamada para execução do Processo de Extração de Características
# Execução de loop de acordo com a quantidade de imagens para processamento

for i in range(0,10): # range = quantidade de pastas disponíveis no bucket 'BK-feature-vector'
    # Chamada de função para seleção automática de imagens
    op, nome_imagem, path_out = selecao_automatica_img(filepaths, contador_imagens)

    # Chamada de função para entrada do arquivo para análise HOG
    image, gray = entrada_imagem(full_path, nome_imagem)

    # Chamada de função para Processamento HOG
    fd, hog_image = processamento_HOG(gray)

    # Chamada de função para plotagem da imagem HOG
    plot_imagem_HOG(image, hog_image)

    # Chamada para armazenar o vetor de características HOG em disco
    text = armazenar_vetor_HOG(fd)

    # Chamada de função para gravar a String em arquivo
    gravar_string_arquivo(path_out,contador_imagens, url)

    # Chamada de função para carregar os dados CSV no Dataframe
    df_hog = carregar_dados_dataframe(path_out, contador_imagens)

    # Chamada de função para processamento do SIFT
    kp, des, sift, img_sift = processar_SIFT(full_path, url, path_out, contador_imagens, nome_imagem, image)

    # Chamada de função para exibição dos características dados gerados SIFT
    exibir_caracateristicas_dados_SIFT()
    
    # Chamada de função para armazenar o vetor de características SIFT em disco
    armazenar_vetor_SIFT()

    # Chamada de função para gerar o cabeçalho para os dados CSV
    df_sift = gerar_data_header(des, path_out, contador_imagens)

    # Chamada de função para processar Momentos Invariantes de Hu
    moments, huMoments, text_value = processar_momentos_HU(path_out, full_path, url, contador_imagens, nome_imagem)

    # Chamada de função para tratar os dados dos Momentos HU e gerar o Dataframe
    df_hu = tratar_dados_HU_dataframe(path_out, contador_imagens)

    # Chamada de função para concatenar os dados dos dataframes de características HOG, SIFT e HU
    df_total = concatenar_dados_dataframes(df_hog, df_sift, df_hu)

    # Função para gerar arquivo CSV com os dados completos
    gerar_arquivo_dados_completos(url, path_out,contador_imagens)

    # Chamada de função para verificar se há colunas com valores faltantes no conjunto de dados
    verificar_valores_faltantes()

    # Chamada de função para preencher os dados com valores "0"
    df_total = preencher_dados_faltantes(df_total)

    # Chamada de função para conferir se há valores faltantes no conjunto de dados - Pós tratamento
    conferir_valores_faltantes()

    # Chamada de função para normalização dos dados (HOG, SIFT e HU)
    normalized_df_total = normalizar_dados(df_total)

    # Chamada de função para aplicar a correlação dos dados normalizados
    corr_df_total = aplicar_correlacao(normalized_df_total)

    # Chamada de função para aplicar PCA - Redução de Dimensionalidade
    pca_instance, pca_data = aplicar_PCA(corr_df_total)

    # Chamada de função para exibir os dados do Dataframe PCA
    df_pca = exibir_dados_PCA(pca_data)

    # Chamada de função para adicionar coluna no final do Dataframe com preenchimento de acordo com a Classe Processada
    df_pca_export = adicionar_coluna_dataframe(df_pca, op, path_out, nome_imagem, contador_imagens)

    # Incremento do contador de pastas
    contador_imagens+=1